# recommendation system

In [50]:
import pandas as pd
import numpy as np
from collections import Counter
import rdflib
from bs4 import BeautifulSoup
import urllib
import re
import shutil
import requests
import copy
import random
import json

In [5]:
movie_df = pd.read_csv('./movies.csv', usecols=['title','genres'])
movie_df.head()

,title,genres
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,Jumanji (1995),Adventure|Children|Fantasy
2,Grumpier Old Men (1995),Comedy|Romance
3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,Father of the Bride Part II (1995),Comedy


In [59]:
movie_title=[' '.join(x.split(' ')[:-1]) for x in movie_df['title'].tolist()]
movie_time=[x.split(' ')[-1][1:5] for x in movie_df['title'].tolist()]
movie_genres=[ x.split('|') for x in movie_df['genres'].tolist()]

In [60]:
for i in range(100):
    print(i,movie_title[i],movie_time[i],movie_genres[i])

0 Toy Story 1995 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
1 Jumanji 1995 ['Adventure', 'Children', 'Fantasy']
2 Grumpier Old Men 1995 ['Comedy', 'Romance']
3 Waiting to Exhale 1995 ['Comedy', 'Drama', 'Romance']
4 Father of the Bride Part II 1995 ['Comedy']
5 Heat 1995 ['Action', 'Crime', 'Thriller']
6 Sabrina 1995 ['Comedy', 'Romance']
7 Tom and Huck 1995 ['Adventure', 'Children']
8 Sudden Death 1995 ['Action']
9 GoldenEye 1995 ['Action', 'Adventure', 'Thriller']
10 American President, The 1995 ['Comedy', 'Drama', 'Romance']
11 Dracula: Dead and Loving It 1995 ['Comedy', 'Horror']
12 Balto 1995 ['Adventure', 'Animation', 'Children']
13 Nixon 1995 ['Drama']
14 Cutthroat Island 1995 ['Action', 'Adventure', 'Romance']
15 Casino 1995 ['Crime', 'Drama']
16 Sense and Sensibility 1995 ['Drama', 'Romance']
17 Four Rooms 1995 ['Comedy']
18 Ace Ventura: When Nature Calls 1995 ['Comedy']
19 Money Train 1995 ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller']
20 Get Shorty 1995 

In [61]:
test=['Toy Story','Jumanji']

In [62]:
def check_genres_in_list(genres_1,genres_2):
    for g in genres_1:
        if g in genres_2:
            return 1
    return 0


def recommend_movies(test,n):
    number_of_all_movies=len(movie_title)

    movie_index=[]
    for t in test:
        try:
            movie_index.append(movie_title.index(t))
        except:
            print('movie has not been found')

    if movie_index==[]:
        print('it seems like you did not input even a movie, it is hard for me to recommmend.')
    else:
        movie_r_time=[]
        movie_r_genres=[]
        for i in movie_index:
            movie_r_time.append(movie_time[i])
            movie_r_genres.append(movie_genres[i])

    genres_counter=Counter()
    for l in movie_r_genres:
        for item in l:
            genres_counter.update({item:1})


    genres_counter=dict(sorted(genres_counter.items(), key=lambda item: item[1],reverse=True))
    movie_r_time=list(set(movie_r_time))

    # print(movie_r_time)
    # print(genres_counter)

    time_r_list=movie_r_time[:n]
    genres_r_list=list(genres_counter)[:n]
    
#     print(time_r_list)
#     print(genres_r_list)

    search_answers_index=[]
    search_answers_number=0
    recommend_reason=-1
    
    
    
    #best: time and genres both pass
    for i in range(number_of_all_movies):
        if movie_time[i] in time_r_list and check_genres_in_list(movie_genres[i],genres_r_list) and movie_title[i] not in test:
            search_answers_number+=1
            search_answers_index.append(i)
            if search_answers_number==n:
                break
  
    if search_answers_index!=[]:
        recommend_reason=0
        return search_answers_index,recommend_reason
    

    
    #only considerate genres    
    for i in range(number_of_all_movies):
        if check_genres_in_list(movie_genres[i],genres_r_list) and movie_title[i] not in test:
            search_answers_number+=1
            search_answers_index.append(i)
            if search_answers_number==n:
                break

    if search_answers_index!=[]:
        recommend_reason=2
        return search_answers_index,recommend_reason    
    
    
    
    
    
    
    
    
    #only considerate time
    for i in range(number_of_all_movies):
        if movie_time[i] in time_r_list and movie_title[i] not in test:
            search_answers_number+=1
            search_answers_index.append(i)
            if search_answers_number==n:
                break

    if search_answers_index!=[]:
        recommend_reason=1
        return search_answers_index,recommend_reason

    else:
        print('no answers have been found')
    return search_answers_index,recommend_reason

In [63]:
recommend_movies(test,3)

([7, 9, 12], 0)

In [64]:
def recommend_answers(search_answers_outcomes):
    search_answers_index,recommend_reason=search_answers_outcomes
    movie_list=[movie_title[x] for x in search_answers_index]
    time_list=[movie_time[x] for x in search_answers_index]
    genres_list=[movie_genres[x] for x in search_answers_index]
    
    recommend_categories=''
    whether_found=(recommend_reason==-1)
    if recommend_reason==-1:
        recommend_categories='have not been found, so no reasons'
    elif recommend_reason==0:
        recommend_categories='all the movies have been published in the same year and the genres are the same'
    elif recommend_reason==1:
        recommend_categories='all the movies have been published in the same year'
    elif recommend_reason==2:
        recommend_categories='all the movies are the same genres'
    return movie_list,time_list,genres_list,recommend_categories,whether_found

In [65]:
recommend_answers(recommend_movies(test,3))

(['Tom and Huck', 'GoldenEye', 'Balto'],
 ['1995', '1995', '1995'],
 [['Adventure', 'Children'],
  ['Action', 'Adventure', 'Thriller'],
  ['Adventure', 'Animation', 'Children']],
 'all the movies have been published in the same year and the genres are the same',
 False)

In [66]:
def generate_answers_for_recommendation(search_answers_outcomes):
    movie_list,time_list,genres_list,recommend_categories,whether_found=search_answers_outcomes
    if whether_found==True:
        return 'no answers have been found, try again.'
    else:
        answer1='Adequate recommendations will be: {}, because {}.'.format(', '.join(movie_list),recommend_categories)
        answer2=' The details about the movies are shown as follows: '
        answer3=' publication date of each movie is {} and the genres of each movie is {}.'.format(', '.join(time_list),genres_list)
        return answer1+answer2+answer3
    

In [67]:
generate_answers_for_recommendation(recommend_answers(recommend_movies(test,3)))

"Adequate recommendations will be: Tom and Huck, GoldenEye, Balto, because all the movies have been published in the same year and the genres are the same. The details about the movies are shown as follows:  publication date of each movie is 1995, 1995, 1995 and the genres of each movie is [['Adventure', 'Children'], ['Action', 'Adventure', 'Thriller'], ['Adventure', 'Animation', 'Children']]."

In [68]:
test=['Who Am I (Kein System Ist Sicher)']

In [69]:
recommend_movies(test,3)

([8340, 8346, 8379], 0)

In [70]:
generate_answers_for_recommendation(recommend_answers(recommend_movies(test,3)))

"Adequate recommendations will be: Freezer, Jack Ryan: Shadow Recruit, Non-Stop, because all the movies have been published in the same year and the genres are the same. The details about the movies are shown as follows:  publication date of each movie is 2014, 2014, 2014 and the genres of each movie is [['Action', 'Thriller'], ['Action', 'Drama', 'Thriller', 'IMAX'], ['Action', 'Mystery', 'Thriller']]."

# Multimedia

In [97]:
#TODO:since we have applied fuzzy search, we could also apply it here

#test1 is an actor
test1='Angelina Jolie'
label1='ACTOR'
#test2 is a film
test2='The Messengers'
label2='MOVIE'

In [98]:
class GraphOperations:
    """
    Handles all operations on the graph:
    - defines graph with fixed path that can be overwritten
    - load graph
    - query graph that returns the answer in a list
    """

    def __init__(self, graph_file="14_graph.nt"):
        self.graph = rdflib.Graph()
        self.load_graph(graph_file)

    def load_graph(self, graph_file):
        print("loading graph ...")
        self.graph.parse(graph_file, format="turtle")
        print("loaded graph successfully!")

    def query(self, message):
        message = message.replace('"""', "").replace("'''", "")
        message = str(message)
                
        temp = [str(s) for s, in self.graph.query(message)]
        # if temp!=[]:
        #     print('temp answer: ',temp)

        return temp

In [100]:
g=GraphOperations()

loading graph ...
loaded graph successfully!


In [105]:
def data_process():
    with open('images.json','r') as f:
        image_data=json.load(f)
    movie_image={}
    cast_image={}
    for d in image_data:
        if len(d['movie'])!=0:
            for m in d['movie']:
                if m in movie_image.keys():
                    movie_image[m].append(d['img'][:-4])
                else:
                    movie_image[m]=[d['img'][:-4]]

        if len(d['cast'])!=0:
            for c in d['cast']:
                if c in cast_image.keys():
                    cast_image[c].append(d['img'][:-4])
                else:
                    cast_image[c]=[d['img'][:-4]]
    with open('movie_image.json','w') as f:
        json.dump(movie_image,f)
    with open('cast_image.json','w') as f:
        json.dump(cast_image,f)                

In [106]:
data_process()
with open('movie_image.json','r') as f:
    movie_image=json.load(f)
with open('cast_image.json','r') as f:
    cast_image=json.load(f)

In [107]:
def search_pictures(name):
    query =f"""
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?id WHERE{{    
            ?entity rdfs:label '{name}'@en.
            ?entity wdt:P345 ?id.
        }}
        LIMIT 1
    """
    imdb_id=g.query(query)
    if imdb_id==[]:
        return '',False
    else:
        return imdb_id[0],True

In [108]:
def generate_answers_for_multimedia(search_pictures_outcomes,label,number):
    imdb_id,whether_found=search_pictures_outcomes
    if whether_found==False:
        return 'NER error, no imdb id found'
    else:
        if label=='ACTOR':
            try:
                cast_length=len(cast_image[imdb_id])
                if number>cast_length:
                    print('number is too large')
                    return ['image:{}'.format(x) for x in cast_image[imdb_id]]
                else:
                    return ['image:{}'.format(cast_image[imdb_id][i]) for i in range(number)]
            except:
                return 'NER error, wrong imdb id or label'
        if label=='MOVIE':
            try:
                movie_length=len(movie_image[imdb_id])                
                if number>movie_length:
                    print('number is too large')
                    return ['image:{}'.format(x) for x in movie_image[imdb_id]]
                else:
                    return ['image:{}'.format(movie_image[imdb_id][i]) for i in range(number)]
            except:
                return 'NER error, wrong imdb id or label'
        return 'NER error, wrong imdb id or label'

In [109]:
def generate_an_answer_for_multimedia(search_pictures_outcomes,label):
    imdb_id,whether_found=search_pictures_outcomes
    if whether_found==False:
        return 'NER error, no imdb id found'
    else:
        if label=='ACTOR':
            try:
                cast_length=len(cast_image[imdb_id])
                return 'image:{}'.format(cast_image[imdb_id][0])
            except:
                return 'NER error, wrong imdb id or label'
        if label=='MOVIE':
            try:
                movie_length=len(movie_image[imdb_id])                
                return 'image:{}'.format(movie_image[imdb_id][0])
            except:
                return 'NER error, wrong imdb id or label'
        return 'NER error, wrong imdb id or label'

In [110]:
#cast_length 2010 for test1
#movie_length 18 for test2

#you could choose whether you want many pictures or a single picture by calling
#generate_answers_for_multimedia or generate_an_answer_for_multimedia

print(generate_an_answer_for_multimedia(search_pictures(test1),label1))
print(generate_an_answer_for_multimedia(search_pictures(test2),label2))


image:0315/rm601699072
image:0094/rm2182646784


# Crowdsourcing

In [14]:
import pandas as pd
import numpy as np

In [15]:
test=['wd:Q11621', 'wdt:P2142']

In [44]:
crowd_df = pd.read_csv("../../data/crowd_data.tsv", sep="\t")
crowd_df[:25]

,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,99%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
1,1,7QT,Is this triple correct or incorrect?,$0.50,2,2133U7HKDLO,Submitted,40,40%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,yes,yes
2,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,98%,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
3,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
4,1,7QT,Is this triple correct or incorrect?,$0.50,5,2134U7HKDMM,Submitted,2,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
5,2,7QT,Is this triple correct or incorrect?,$0.50,6,2134U7HKDMM,Submitted,2,70%,wd:Q603545,wdt:P2142,4300000,2.0,INCORRECT,NaN,NaN
6,2,7QT,Is this triple correct or incorrect?,$0.50,7,2133ICYWE97,Submitted,120,99%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
7,2,7QT,Is this triple correct or incorrect?,$0.50,8,928UJANWZ12,Submitted,60,98%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
8,2,7QT,Is this triple correct or incorrect?,$0.50,9,1726JMZQW,Submitted,90,70%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
9,2,7QT,Is this triple correct or incorrect?,$0.50,10,2133U7HKDLO,Submitted,40,40%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,yes,yes


In [45]:
def convert_percentage_to_float(percentage_string):
    """

    Convert a percentage string to a float.

    For example, '99%' becomes 0.99.

    """

    if isinstance(percentage_string, str):
        return float(percentage_string.strip("%")) / 100

    return percentage_string


crowd_df["LifetimeApprovalRate"] = crowd_df["LifetimeApprovalRate"].apply(
    convert_percentage_to_float
)

In [46]:
crowd_df.describe()

,HITId,AssignmentId,WorkTimeInSeconds,LifetimeApprovalRate,AnswerID
count,305.000000,305.000000,305.000000,305.000000,304.000000
mean,31.000000,153.000000,108.800000,0.747443,1.473684
std,17.635752,88.190136,95.548706,0.204038,0.500130
min,1.000000,1.000000,2.000000,0.400000,1.000000
25%,16.000000,77.000000,4.000000,0.690000,1.000000
50%,31.000000,153.000000,120.000000,0.800000,1.000000
75%,46.000000,229.000000,200.000000,0.980000,2.000000
max,61.000000,305.000000,337.000000,0.990000,2.000000


In [47]:
crowd_df = crowd_df.drop(crowd_df[crowd_df.LifetimeApprovalRate < 0.5].index)
crowd_df = crowd_df.drop(crowd_df[crowd_df.WorkTimeInSeconds < 40].index)

In [49]:
crowd_df.describe()

,HITId,AssignmentId,WorkTimeInSeconds,LifetimeApprovalRate,AnswerID
count,183.000000,183.000000,183.000000,183.000000,183.000000
mean,31.000000,152.251366,174.065574,0.881257,1.508197
std,17.655121,88.228115,66.240206,0.100444,0.501304
min,1.000000,1.000000,50.000000,0.700000,1.000000
25%,16.000000,76.500000,120.000000,0.800000,1.000000
50%,31.000000,152.000000,200.000000,0.850000,2.000000
75%,46.000000,227.500000,240.000000,0.980000,2.000000
max,61.000000,303.000000,337.000000,0.990000,2.000000


In [50]:
crowd_df = crowd_df[
    ["HITId", "Input1ID", "Input2ID", "Input3ID", "AnswerLabel"]
].reset_index(drop=True)
crowd_df[:22]

,HITId,Input1ID,Input2ID,Input3ID,AnswerLabel
0,1,wd:Q11621,wdt:P2142,792910554,CORRECT
1,1,wd:Q11621,wdt:P2142,792910554,INCORRECT
2,1,wd:Q11621,wdt:P2142,792910554,CORRECT
3,2,wd:Q603545,wdt:P2142,4300000,CORRECT
4,2,wd:Q603545,wdt:P2142,4300000,CORRECT
5,2,wd:Q603545,wdt:P2142,4300000,CORRECT
6,3,wd:Q16911843,wdt:P577,2014-01-18,INCORRECT
7,3,wd:Q16911843,wdt:P577,2014-01-18,CORRECT
8,3,wd:Q16911843,wdt:P577,2014-01-18,CORRECT
9,4,wd:Q132863,wdt:P2142,969023261,INCORRECT


In [51]:
def process_crowd(crowd_df):
    crowd_numpy = crowd_df.to_numpy().tolist()

    question_id = 0
    rdf_turple = []
    people_views = []
    correct_number = 0
    incorrect_number = 0

    for survey_data in crowd_numpy:
        if survey_data[0] != question_id:
            people_views.append([correct_number, incorrect_number])
            correct_number = 0
            incorrect_number = 0
            question_id += 1
            rdf_turple.append([survey_data[1], survey_data[2], survey_data[3]])
            if survey_data[4] == "CORRECT":
                correct_number += 1
            else:
                incorrect_number += 1
        else:
            if survey_data[4] == "CORRECT":
                correct_number += 1
            else:
                incorrect_number += 1

    people_views.append([correct_number, incorrect_number])
    people_views = people_views[1:]
    people_views = np.array(people_views, dtype=int)

    people_number = sum(people_views[0])
    evaluations_number = len(rdf_turple)

    #     print(rdf_turple)
    #     print(len(rdf_turple))
    #     print(people_views)
    #     print(len(people_views))

    # cal k in total

    p1 = np.sum(people_views[:, 0]) / (people_number * evaluations_number)
    p2 = np.sum(people_views[:, 1]) / (people_number * evaluations_number)
    P = (np.sum(np.square(people_views), axis=1) - people_number) / (
        people_number * (people_number - 1)
    )
    average_P = np.sum(P) / evaluations_number
    average_p_e = np.square(p1) + np.square(p2)
    total_k = (average_P - average_p_e) / (1 - average_p_e)

    return rdf_turple, people_views, total_k

In [54]:
def generate_answers_for_crowdsourcing(crowd_df, test):
    rdf_turple, people_views, total_k = process_crowd(crowd_df)

    #     print(rdf_turple)
    #     print(people_views)

    for i in range(len(rdf_turple)):
        if rdf_turple[i][0] == test[0] and rdf_turple[i][1] == test[1]:
            return rdf_turple[i][2], people_views[i], cal_k(people_views[i]), total_k


def cal_k(people_approvals):
    people_number = 3
    evaluations_number = 1
    p1 = (people_approvals[0]) / (people_number * evaluations_number)
    p2 = (people_approvals[1]) / (people_number * evaluations_number)
    P = (np.sum(np.square(people_approvals)) - people_number) / (
        people_number * (people_number - 1)
    )

    average_P = P / evaluations_number
    average_p_e = np.square(p1) + np.square(p2)

    k = (average_P - average_p_e) / (1 - average_p_e)
    return k

In [55]:
generate_answers_for_crowdsourcing(crowd_df, test)
# answer, people's views(3 for voting and 1 for rejecting), k for this question, k for the whole graph

('792910554', array([2, 1]), -0.5000000000000001, 0.16877838871623255)